In [23]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
from tqdm import tqdm
from dateutil.parser import parse

In [24]:
# Connect with GSC
key = "apis-test-346309-4913a5dc3905.json"
def connect(key):
  scope = ['https://www.googleapis.com/auth/webmasters']
  credentials = service_account.Credentials.from_service_account_file(key, scopes=scope)
  service = build('searchconsole','v1',credentials=credentials)
  return service

service = connect(key)

In [41]:
import requests
from bs4 import BeautifulSoup

wunder = requests.get("https://crossword-solver.io/sitemaps_knotilex/clues-latest.xml")
parcala = BeautifulSoup(wunder.content, "xml")

urls_from_xml = []

loc_tags = parcala.find_all('loc')

for loc in loc_tags:
    urls_from_xml.append(loc.get_text()) 

In [42]:
siteurl = "sc-domain:crossword-solver.io"
today = str(datetime.date.today().strftime('%Y-%m-%d'))
number_urls = 2000

verdict = []
coverageState = []
robotsTxtState = []
indexingState= []
sitemap = []
pageFetchState = []
lastCrawlTime = []


for url in urls_from_xml[0:number_urls]:
    print("Making request for: " + url)
    request = {
    "inspectionUrl": url,
    "languageCode": "en",
    "siteUrl": siteurl
    } 
    try:
        response = service.urlInspection().index().inspect(body=request).execute()

        #valores
        inspectionResult = response['inspectionResult']

        verdict.append(inspectionResult['indexStatusResult']['verdict'])
        coverageState.append(inspectionResult['indexStatusResult']['coverageState'])
        robotsTxtState.append(inspectionResult['indexStatusResult']['robotsTxtState'])
        indexingState.append(inspectionResult['indexStatusResult']['indexingState'])
        pageFetchState.append(inspectionResult['indexStatusResult']['pageFetchState'])
        if 'sitemap' in inspectionResult['indexStatusResult']:
          sitemap.append(inspectionResult['indexStatusResult']['sitemap'])
        else:
          sitemap.append("N/A")
        if 'lastCrawlTime' in inspectionResult['indexStatusResult']:
          time = parse(inspectionResult['indexStatusResult']['lastCrawlTime'])
          lastCrawlTime.append(time.date())
        else:
          lastCrawlTime.append("Never")
    except Exception as e:
        print(e)
        verdict.append('ERROR')
        coverageState.append('ERROR')
        robotsTxtState.append('ERROR')
        indexingState.append('ERROR')
        pageFetchState.append('ERROR')
        sitemap.append('ERROR')
        lastCrawlTime.append('ERROR')

df = pd.DataFrame({
  'URL': urls_from_xml[0:number_urls],
  'Status': verdict,
  'Indexing_State': indexingState,
  'Coverage_Status': coverageState,
  'Robots.txt_status': robotsTxtState,
  'Page_Fetch_Status': pageFetchState,
  'On_the_Sitemap': sitemap,  
  'Last_Crawl': lastCrawlTime,
  'GSC_API_day_request': [today] * number_urls
    })

df.to_csv('test3_sitemap.csv',encoding='utf-8')

Making request for: https://crossword-solver.io/clue/%27glad-that%27s-settled%21%27/
Making request for: https://crossword-solver.io/clue/actor-willem/
Making request for: https://crossword-solver.io/clue/big-name-in-blue-jeans/
Making request for: https://crossword-solver.io/clue/approximately/
Making request for: https://crossword-solver.io/clue/anaheim-attraction/
Making request for: https://crossword-solver.io/clue/brief-moment/
Making request for: https://crossword-solver.io/clue/challenges-requiring-nerve/
Making request for: https://crossword-solver.io/clue/droop/
Making request for: https://crossword-solver.io/clue/first-name-in-fashion/
Making request for: https://crossword-solver.io/clue/vacation-rental/
Making request for: https://crossword-solver.io/clue/be-in-session/
Making request for: https://crossword-solver.io/clue/bicker/
Making request for: https://crossword-solver.io/clue/alliance-acronym/
Making request for: https://crossword-solver.io/clue/%27leaves-of-grass%27-n